In [ ]:
import telebot

f=open("/authentication/account.txt","r")
lines=f.readlines()
API_TOKEN = lines[0]
f.close()

bot = telebot.TeleBot(API_TOKEN)


# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am EchoBot.
I am here to echo your kind words back to you. Just say anything nice and I'll say the exact same thing to you!\
""")


# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)


bot.infinity_polling()

2025-04-16 22:15:05,106 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-04-16 22:15:05,107 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
